

# **📘 Recap**

### **What is an HMM (Hidden Markov Model)?**

A **Hidden Markov Model** is a statistical model used when:

* Something is happening **behind the scenes** (hidden states) — we cannot see these states directly.
* We only observe **outputs** (visible signals).
* The hidden states follow the **Markov property** (next state depends only on the current state).
* Each state emits observations with some probability.

Think of it like:

> You see the symptoms (observations) but not the actual health condition (hidden state).

---

# **What the Notebook is Doing**

The notebook **teaches HMMs through a fun toy example**:

### ✅ **1. A motivating scenario**

A friend travels between cities and sends you a selfie each day.
You want to **guess which city they are in** based only on:

* the selfie (observation)
* knowledge of how they usually move between cities (transition probabilities)

This real-life analogy is modeled as an HMM.

---

### ✅ **2. Explaining HMM structure**

The notebook walks through:

* **Hidden states** → the city your friend is in
* **Observations** → the type of selfie you receive
* **Transition probabilities** → chance of moving from one city to another
* **Emission probabilities** → chance of getting a certain kind of selfie from a city

---

### ✅ **3. HMM assumptions**

The two classic assumptions:

1. **Markov property**: next city depends only on current city
2. **Emission independence**: selfie depends only on current city, not previous days

---

### ✅ **4. HMM components**

The notebook introduces all the key parameters:

* **π (initial distribution)** – where the journey starts
* **A (transition matrix)** – how likely it is to move between states
* **B (emission matrix)** – how likely a state is to emit each observation




# Fun with Hidden Markov Models
*by Loren Lugosch*

This notebook introduces the Hidden Markov Model (HMM), a simple model for sequential data.

We will see:
- what an HMM is and when you might want to use it;
- the so-called "three problems" of an HMM; and
- how to implement an HMM in PyTorch.

(The code in this notebook can also be found at https://github.com/lorenlugosch/pytorch_HMM.)

A hypothetical scenario
------

To motivate the use of HMMs, imagine that you have a friend who gets to do a lot of travelling. Every day, this jet-setting friend sends you a selfie from the city they’re in, to make you envious.

<center>

![Diagram of a traveling friend sending selfies](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/selfies.png?raw=true)
</center>





How would you go about guessing which city the friend is in each day, just by looking at the selfies?

If the selfie contains a really obvious landmark, like the Eiffel Tower, it will be easy to figure out where the photo was taken. If not, it will be a lot harder to infer the city.

But we have a clue to help us: the city the friend is in each day is not totally random. For example, the friend will probably remain in the same city for a few days to sightsee before flying to a new city.

## The HMM setup

The hypothetical scenario of the friend travelling between cities and sending you selfies can be modeled using an HMM.


An HMM models a system that is in a particular state at any given time and produces an output that depends on that state.

At each timestep or clock tick, the system randomly decides on a new state and jumps into that state. The system then randomly generates an observation. The states are "hidden": we can't observe them. (In the cities/selfies analogy, the unknown cities would be the hidden states, and the selfies would be the observations.)

Let's denote the sequence of states as $\mathbf{z} = \{z_1, z_2, \dots, z_T \}$, where each state is one of a finite set of $N$ states, and the sequence of observations as $\mathbf{x} = \{x_1, x_2, \dots, x_T\}$. The observations could be discrete, like letters, or real-valued, like audio frames.

<center>

![Diagram of an HMM for three timesteps](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/hmm.png?raw=true)
</center>

An HMM makes two key assumptions:
- **Assumption 1:** The state at time $t$ depends *only* on the state at the previous time $t-1$.
- **Assumption 2:** The output at time $t$ depends *only* on the state at time $t$.

These two assumptions make it possible to efficiently compute certain quantities that we may be interested in.

## Components of an HMM
An HMM has three sets of trainable parameters.
  


- The **transition model** is a square matrix $A$, where $A_{s, s'}$ represents $p(z_t = s|z_{t-1} = s')$, the probability of jumping from state $s'$ to state $s$.

- The **emission model** $b_s(x_t)$ tells us $p(x_t|z_t = s)$, the probability of generating $x_t$ when the system is in state $s$. For discrete observations, which we will use in this notebook, the emission model is just a lookup table, with one row for each state, and one column for each observation. For real-valued observations, it is common to use a Gaussian mixture model or neural network to implement the emission model.

- The **state priors** tell us $p(z_1 = s)$, the probability of starting in state $s$. We use $\pi$ to denote the vector of state priors, so $\pi_s$ is the state prior for state $s$.

Let's program an HMM class in PyTorch.

In [16]:
import torch
import numpy as np

class HMM(torch.nn.Module):
  """
  Hidden Markov Model with discrete observations.
  """
  def __init__(self, M, N):
    super(HMM, self).__init__()
    self.M = M # number of possible observations
    self.N = N # number of states

    # A
    self.transition_model = TransitionModel(self.N)

    # b(x_t)
    self.emission_model = EmissionModel(self.N,self.M)

    # pi
    self.unnormalized_state_priors = torch.nn.Parameter(torch.randn(self.N))

    # use the GPU
    self.is_cuda = torch.cuda.is_available()
    if self.is_cuda: self.cuda()

class TransitionModel(torch.nn.Module):
  def __init__(self, N):
    super(TransitionModel, self).__init__()
    self.N = N
    self.unnormalized_transition_matrix = torch.nn.Parameter(torch.randn(N,N))

class EmissionModel(torch.nn.Module):
  def __init__(self, N, M):
    super(EmissionModel, self).__init__()
    self.N = N
    self.M = M
    self.unnormalized_emission_matrix = torch.nn.Parameter(torch.randn(N,M))

To sample from the HMM, we start by picking a random initial state from the state prior distribution.

Then, we sample an output from the emission distribution, sample a transition from the transition distribution, and repeat.

(Notice that we pass the unnormalized model parameters through a softmax function to make them into probabilities.)




### **1. Why are the transition, emission, and prior parameters named “unnormalized”?**

**Answer:**
They are stored as **raw learnable tensors**, not probabilities yet.
Before using them in Forward/Backward/Viterbi, we typically apply a **softmax** to convert them into valid probability distributions:

* `pi = softmax(unnormalized_state_priors)`
* `A = softmax(unnormalized_transition_matrix, dim=1)`
* `B = softmax(unnormalized_emission_matrix, dim=1)`

This allows PyTorch to optimize them freely without the constraint of summing to 1 during training.

---

### **2. Why use `torch.nn.Parameter` for all HMM matrices?**

**Answer:**
`nn.Parameter` tells PyTorch:
**“This tensor should be updated by gradient descent.”**
So transition matrix, emission matrix, and state priors become **learnable parameters** when training with `loss.backward()`.

If they were plain tensors, PyTorch would *not* update them.

---

### **3. Why are the parameters initialized with `torch.randn()` instead of fixed values?**

**Answer:**
`torch.randn()` provides **random normal initialization**, which:

* avoids symmetry in optimization
* gives the model flexibility to learn any distribution
* works well with softmax-based probability normalization

Since HMM will be trained, random initialization is appropriate and common practice.

---

### **4. Why is CUDA checked inside the HMM constructor?**

**Answer:**
Because HMM contains learnable parameters (`A`, `B`, `pi`), moving the main model to GPU automatically moves all submodules:

```python
if self.is_cuda:
    self.cuda()
```

This ensures:

* all computation (forward, backward, Viterbi) is **GPU-accelerated**
* no mismatch between CPU and GPU tensors, which would lead to runtime errors

---

### **5. Why does the model separate TransitionModel and EmissionModel into different classes?**

**Answer:**
This design makes your HMM **modular and clean**:

* TransitionModel handles **state → state** probabilities
* EmissionModel handles **state → observation** probabilities
* Each can be extended or replaced independently, e.g.,

  * add dropout
  * switch to neural emissions
  * visualize parameters
  * plug in learned embeddings

It also makes the code easier to read and demo, since each part of the HMM is encapsulated.




In [17]:
def sample(self, T=10):
  state_priors = torch.nn.functional.softmax(self.unnormalized_state_priors, dim=0)
  transition_matrix = torch.nn.functional.softmax(self.transition_model.unnormalized_transition_matrix, dim=0)
  emission_matrix = torch.nn.functional.softmax(self.emission_model.unnormalized_emission_matrix, dim=1)

  # sample initial state
  z_t = torch.distributions.categorical.Categorical(state_priors).sample().item()
  z = []; x = []
  z.append(z_t)
  for t in range(0,T):
    # sample emission
    x_t = torch.distributions.categorical.Categorical(emission_matrix[z_t]).sample().item()
    x.append(x_t)

    # sample transition
    z_t = torch.distributions.categorical.Categorical(transition_matrix[:,z_t]).sample().item()
    if t < T-1: z.append(z_t)

  return x, z

# Add the sampling method to our HMM class
HMM.sample = sample



### **1. Why do we apply softmax to the priors, transition matrix, and emission matrix before sampling?**

**Answer:**
The parameters stored in the model are **unnormalized logits**.
Softmax converts them into valid probability distributions:

* `state_priors` → probabilities of initial states
* `transition_matrix` → probabilities of moving from one state to another
* `emission_matrix` → probabilities of emitting observations

Without softmax, sampling would fail because the values would not sum to 1.

---

### **2. Why does the transition sampling use `transition_matrix[:, z_t]` instead of `transition_matrix[z_t]`?**

**Answer:**
Because the transition matrix is stored so that:

* **Columns** represent the *next-state* distribution
* **Rows** represent the *from-state*

So `transition_matrix[:, z_t]` means:

> “Given we are currently in state z_t, sample the next state based on the column z_t.”

Many students expect row-wise transitions; your code uses column-wise.
This is perfectly valid as long as consistency is maintained.

---

### **3. Why do we sample emissions before sampling the next hidden state?**

**Answer:**
This follows the generative process of an HMM:

1. Pick initial hidden state
2. Emit observation from that state
3. Transition to next hidden state
4. Repeat

The order **emission → transition** reflects the standard generative structure:

```
z_t → x_t
z_t → z_(t+1)
```

Both orders are allowed mathematically, but this one matches common textbooks.

---

### **4. Why does the method return both `x` (observations) and `z` (hidden states)?**

**Answer:**
Because for a demo:

* `x` shows what a **generated observation sequence** looks like
* `z` allows you to **demonstrate the underlying hidden path**

This is extremely helpful when teaching:

* how HMMs generate data
* how Viterbi decoding tries to recover z from x
* how transition and emission probabilities shape the sequence

It’s a great choice for visualization.

---

### **5. Why is the initial state added to `z` before the sampling loop, but later states added inside the loop?**

**Answer:**
Because the first state is sampled **before** time-step iteration:

```python
z_t = Categorical(state_priors).sample()
z.append(z_t)
```

Then each loop iteration emits an observation and transitions to the next state.
The condition:

```python
if t < T-1:
    z.append(z_t)
```

avoids adding an extra next-state after the final iteration.

This keeps `len(z) = T`, matching the length of `x`.


Let's try hard-coding an HMM for generating fake words. (We'll also add some helper functions for encoding and decoding strings.)

We will assume that the system has one state for generating vowels and one state for generating consonants, and the transition matrix has 0s on the diagonal---in other words, the system cannot stay in the vowel state or the consonant state for one than one timestep; it has to switch.

Since we pass the transition matrix through a softmax, to get 0s we set the unnormalized parameter values to $-\infty$.

In [18]:
import string
alphabet = string.ascii_lowercase

def encode(s):
  """
  Convert a string into a list of integers
  """
  x = [alphabet.index(ss) for ss in s]
  return x

def decode(x):
  """
  Convert list of ints to string
  """
  s = "".join([alphabet[xx] for xx in x])
  return s

# Initialize the model
model = HMM(M=len(alphabet), N=2)

# Hard-wiring the parameters!
# Let state 0 = consonant, state 1 = vowel
for p in model.parameters():
    p.requires_grad = False # needed to do lines below
model.unnormalized_state_priors[0] = 0.    # Let's start with a consonant more frequently
model.unnormalized_state_priors[1] = -0.5
print("State priors:", torch.nn.functional.softmax(model.unnormalized_state_priors, dim=0))

# In state 0, only allow consonants; in state 1, only allow vowels
vowel_indices = torch.tensor([alphabet.index(letter) for letter in "aeiou"])
consonant_indices = torch.tensor([alphabet.index(letter) for letter in "bcdfghjklmnpqrstvwxyz"])
model.emission_model.unnormalized_emission_matrix[0, vowel_indices] = -np.inf
model.emission_model.unnormalized_emission_matrix[1, consonant_indices] = -np.inf
print("Emission matrix:", torch.nn.functional.softmax(model.emission_model.unnormalized_emission_matrix, dim=1))

# Only allow vowel -> consonant and consonant -> vowel
model.transition_model.unnormalized_transition_matrix[0,0] = -np.inf  # consonant -> consonant
model.transition_model.unnormalized_transition_matrix[0,1] = 0.       # vowel -> consonant
model.transition_model.unnormalized_transition_matrix[1,0] = 0.       # consonant -> vowel
model.transition_model.unnormalized_transition_matrix[1,1] = -np.inf  # vowel -> vowel
print("Transition matrix:", torch.nn.functional.softmax(model.transition_model.unnormalized_transition_matrix, dim=0))



State priors: tensor([0.6225, 0.3775], device='cuda:0')
Emission matrix: tensor([[0.0000, 0.2163, 0.0250, 0.0134, 0.0000, 0.0153, 0.0334, 0.0712, 0.0000,
         0.0456, 0.0131, 0.0594, 0.0249, 0.0363, 0.0000, 0.1069, 0.0353, 0.0116,
         0.0144, 0.0151, 0.0000, 0.0340, 0.1091, 0.0645, 0.0422, 0.0132],
        [0.2717, 0.0000, 0.0000, 0.0000, 0.2163, 0.0000, 0.0000, 0.0000, 0.2508,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0335, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.2277, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
Transition matrix: tensor([[0., 1.],
        [1., 0.]], device='cuda:0')




### **1. Why do we set `requires_grad = False` before manually assigning parameters?**

**Answer:**
`requires_grad=False` is needed because PyTorch **blocks in-place modifications** on tensors that require gradients.
Since you’re *manually hard-wiring* transition/emission probabilities (e.g., setting some to `-inf`), you must disable gradients:

```python
for p in model.parameters():
    p.requires_grad = False
```

Otherwise PyTorch will throw errors like:
**“a leaf Variable that requires grad is being used in an in-place operation.”**

---

### **2. Why is `-np.inf` used in the emission and transition matrices?**

**Answer:**
`-inf` turns into **probability 0** after softmax:

```
softmax(-inf) = 0
```

This is a clean way to *completely forbid* certain transitions or emissions.

Examples in your model:

* Consonant state → cannot emit vowels
* Vowel state → cannot emit consonants
* Consonant → consonant transitions forbidden
* Vowel → vowel transitions forbidden

It enforces the structure:

```
consonant ↔ vowel alternation only
```

Perfect for demonstrating a rule-based HMM.

---

### **3. Why do we convert letters to numbers using `encode()` and `decode()`?**

**Answer:**
HMMs operate on **indices**, not characters.

`encode()` maps a string like `"hello"` to integer indices:
`[7, 4, 11, 11, 14]`

`decode()` converts predicted integer sequences back to readable strings.

This mapping allows:

* emission matrix to index observations
* sampling to produce integer sequences
* decoding to show human-interpretable letters

It’s the standard preprocessing step for discrete HMMs.

---

### **4. Why is the emission matrix built using vowel indices and consonant indices?**

**Answer:**
You define:

* **State 0 = consonant state**
* **State 1 = vowel state**

Then enforce:

```python
state 0 emits ONLY consonants  
state 1 emits ONLY vowels
```

by setting impossible emissions to `-inf`.

This creates a **linguistically meaningful HMM** where sampling produces alternating consonant–vowel patterns (like “b a l o n e”).

It’s also excellent pedagogically because students immediately see how **emission constraints shape outputs**.

---

### **5. Why does the transition matrix forbid same-type transitions (C→C and V→V)?**

**Answer:**
You set:

```python
C→C = -inf  
V→V = -inf  
C→V = 0  
V→C = 0
```

After softmax, the model becomes:

```
C → V  with probability 1  
V → C  with probability 1
```

This enforces *strict alternation*:

```
consonant → vowel → consonant → vowel → ...
```

This cleanly demonstrates:

* how transition probabilities shape hidden sequences
* how to hard-code structural linguistic patterns
* how HMM sampling behaves under deterministic transitions



Try sampling from our hard-coded model:


In [19]:
# Sample some outputs
for _ in range(4):
  sampled_x, sampled_z = model.sample(T=5)
  print("x:", decode(sampled_x))
  print("z:", sampled_z)

x: ipeje
z: [1, 0, 1, 0, 1]
x: wiqiw
z: [0, 1, 0, 1, 0]
x: babix
z: [0, 1, 0, 1, 0]
x: hiley
z: [0, 1, 0, 1, 0]


## The Three Problems

In a [classic tutorial](https://www.cs.cmu.edu/~cga/behavior/rabiner1.pdf) on HMMs, Lawrence Rabiner describes "three problems" that need to be solved before you can effectively use an HMM. They are:
- Problem 1: How do we efficiently compute $p(\mathbf{x})$?
- Problem 2: How do we find the most likely state sequence $\mathbf{z}$ that could have generated the data?
- Problem 3: How do we train the model?

In the rest of the notebook, we will see how to solve each problem and implement the solutions in PyTorch.

### Problem 1: How do we compute $p(\mathbf{x})$?


#### *Why?*
Why might we care about computing $p(\mathbf{x})$? Here's two reasons.
* Given two HMMs, $\theta_1$ and $\theta_2$, we can compute the likelihood of some data $\mathbf{x}$ under each model, $p_{\theta_1}(\mathbf{x})$ and $p_{\theta_2}(\mathbf{x})$, to decide which model is a better fit to the data.

  (For example, given an HMM for English speech and an HMM for French speech, we could compute the likelihood given each model, and pick the model with the higher likelihood to infer whether the person is speaking English or French.)
* Being able to compute $p(\mathbf{x})$ gives us a way to train the model, as we will see later.

#### *How?*
Given that we want $p(\mathbf{x})$, how do we compute it?

We've assumed that the data is generated by visiting some sequence of states $\mathbf{z}$ and picking an output $x_t$ for each $z_t$ from the emission distribution $p(x_t|z_t)$. So if we knew $\mathbf{z}$, then the probability of $\mathbf{x}$ could be computed as follows:

$$p(\mathbf{x}|\mathbf{z}) = \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

However, we don't know $\mathbf{z}$; it's hidden. But we do know the probability of any given $\mathbf{z}$, independent of what we observe. So we could get the probability of $\mathbf{x}$ by summing over the different possibilities for $\mathbf{z}$, like this:

$$p(\mathbf{x}) = \sum_{\mathbf{z}} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z}) = \sum_{\mathbf{z}} \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

The problem is: if you try to take that sum directly, you will need to compute $N^T$ terms. This is impossible to do for anything but very short sequences. For example, let's say the sequence is of length $T=100$ and there are $N=2$ possible states. Then we would need to check $N^T = 2^{100} \approx 10^{30}$ different possible state sequences.

We need a way to compute $p(\mathbf{x})$ that doesn't require us to explicitly calculate all $N^T$ terms. For this, we use the forward algorithm.

________

<u><b>The Forward Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha_{s,1} := b_s(x_1) \cdot \pi_s$
>
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\alpha_{s,t} := b_s(x_t) \cdot \underset{s'}{\sum} A_{s, s'} \cdot \alpha_{s',t-1} $
>
> $p(\mathbf{x}) := \underset{s}{\sum} \alpha_{s,T}$\
> return $p(\mathbf{x})$
________


The forward algorithm is much faster than enumerating all $N^T$ possible state sequences: it requires only $O(N^2T)$ operations to run, since each step is mostly multiplying the vector of forward variables by the transition matrix. (And very often we can reduce that complexity even further, if the transition matrix is sparse.)

There is one practical problem with the forward algorithm as presented above: it is prone to underflow due to multiplying a long chain of small numbers, since probabilities are always between 0 and 1. Instead, let's do everything in the log domain. In the log domain, a multiplication becomes a sum, and a sum becomes a [logsumexp](https://lorenlugosch.github.io/posts/2020/06/logsumexp/).  

________

<u><b>The Forward Algorithm (Log Domain)</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\text{log }\alpha_{s,1} := \text{log }b_s(x_1) + \text{log }\pi_s$
>
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\text{log }\alpha_{s,t} := \text{log }b_s(x_t) +  \underset{s'}{\text{logsumexp}} \left( \text{log }A_{s, s'} + \text{log }\alpha_{s',t-1} \right)$
>
> $\text{log }p(\mathbf{x}) := \underset{s}{\text{logsumexp}} \left( \text{log }\alpha_{s,T} \right)$\
> return $\text{log }p(\mathbf{x})$
________

Now that we have a numerically stable version of the forward algorithm, let's implement it in PyTorch.

In [20]:
def HMM_forward(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)

  Compute log p(x) for each example in the batch.
  T = length of each example
  """
  if self.is_cuda:
  	x = x.cuda()
  	T = T.cuda()

  batch_size = x.shape[0]; T_max = x.shape[1]
  log_state_priors = torch.nn.functional.log_softmax(self.unnormalized_state_priors, dim=0)
  log_alpha = torch.zeros(batch_size, T_max, self.N)
  if self.is_cuda: log_alpha = log_alpha.cuda()

  log_alpha[:, 0, :] = self.emission_model(x[:,0]) + log_state_priors
  for t in range(1, T_max):
    log_alpha[:, t, :] = self.emission_model(x[:,t]) + self.transition_model(log_alpha[:, t-1, :])

  # Select the sum for the final timestep (each x may have different length).
  log_sums = log_alpha.logsumexp(dim=2)
  log_probs = torch.gather(log_sums, 1, T.view(-1,1) - 1)
  return log_probs

def emission_model_forward(self, x_t):
  log_emission_matrix = torch.nn.functional.log_softmax(self.unnormalized_emission_matrix, dim=1)
  out = log_emission_matrix[:, x_t].transpose(0,1)
  return out

def transition_model_forward(self, log_alpha):
  """
  log_alpha : Tensor of shape (batch size, N)
  Multiply previous timestep's alphas by transition matrix (in log domain)
  """
  log_transition_matrix = torch.nn.functional.log_softmax(self.unnormalized_transition_matrix, dim=0)

  # Matrix multiplication in the log domain
  out = log_domain_matmul(log_transition_matrix, log_alpha.transpose(0,1)).transpose(0,1)
  return out

def log_domain_matmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Normally, a matrix multiplication
	computes out_{i,j} = sum_k A_{i,k} x B_{k,j}

	A log domain matrix multiplication
	computes out_{i,j} = logsumexp_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	# log_A_expanded = torch.stack([log_A] * p, dim=2)
	# log_B_expanded = torch.stack([log_B] * m, dim=0)
    # fix for PyTorch > 1.5 by egaznep on Github:
	log_A_expanded = torch.reshape(log_A, (m,n,1))
	log_B_expanded = torch.reshape(log_B, (1,n,p))

	elementwise_sum = log_A_expanded + log_B_expanded
	out = torch.logsumexp(elementwise_sum, dim=1)

	return out

TransitionModel.forward = transition_model_forward
EmissionModel.forward = emission_model_forward
HMM.forward = HMM_forward



### **1. Why do we use the Forward algorithm recurrence instead of brute-forcing all hidden-state sequences?**

**Significance in code:**
The line

```python
log_alpha[:, t, :] = emission(...) + transition_model(log_alpha[:, t-1])
```

compresses an exponential number of state paths into a **single dynamic-programming step**.

**Significance for HMM:**
An HMM with `N` states and sequence length `T` has **N^T** possible hidden paths.
Brute forcing all paths is impossible.

The Forward algorithm *efficiently sums over all possible hidden paths* while preserving exact probabilities:

[
p(x_{1:T}) = \sum_{all ; z} p(x, z)
]

This is the **core reason** why HMMs are computationally feasible.

---

### **2. Why do we work in log space instead of normal probability space?**

**Significance in code:**
All transitions and emissions go through:

```python
log_softmax(...)
```

**Significance for HMM:**
HMMs multiply many probabilities. Even valid sequences may have likelihoods like:

[
10^{-40}, 10^{-80}, 10^{-150}
]

These collapse to zero in floating point arithmetic.

Log space transforms:

* multiplication → addition
* summation → logsumexp
* extremely tiny probabilities → stable numbers

The HMM Forward algorithm **relies on this stability** to produce meaningful likelihoods.

If you do not use log space, the HMM forward pass becomes useless for moderate-length sequences.

---

### **3. Why do we compute emission probabilities using indexing (`emission_model(x[:, t])`) instead of full matrix multiplication?**

**Significance in code:**
You only fetch emission probabilities **for the observed symbol** at time `t`.

**Significance for HMM:**
The HMM emission model defines:

[
p(x_t | z_t)
]

It is **conditional on the observation**, not on every possible observation.
By indexing directly:

```python
emission_matrix[:, x_t]
```

you implement the exact HMM rule:
*“Which states could have emitted this specific observed symbol?”*

This avoids unnecessary work and respects the structure of discrete emission HMMs.

---

### **4. Why do we use a log-domain matrix multiplication instead of regular matrix multiplication for transitions?**

**Significance in code:**
Your custom method:

```python
log_domain_matmul(log_A, log_B)
```

computes:

[
\log \sum_i \exp(\log A_{i \to j} + \log \alpha_{t-1}(i))
]

**Significance for HMM:**
The Forward algorithm needs to **sum over all previous states**:

[
\alpha_t(j) = b_j(x_t) \sum_i \alpha_{t-1}(i) A_{i \to j}
]

Regular matrix multiplication performs **products**, not log-sum-exp.
Log-domain matmul implements the *exact HMM update rule* in a numerically safe way.

Without this, you are not executing the true HMM Forward algorithm.

---

### **5. Why do we use `logsumexp` over hidden states at the final timestep instead of taking a max or using the last α value directly?**

**Significance in code:**

```python
log_sums = log_alpha.logsumexp(dim=2)
```

**Significance for HMM:**
For an HMM, the probability of the entire observed sequence is:

[
p(x_{1:T}) = \sum_{states ; i} \alpha_T(i)
]

This is a **sum**, not a maximum.
Taking a max would correspond to the **Viterbi algorithm**, which finds:

* most likely hidden path
  not
* total likelihood of the observations.

Forward algorithm = **sum over all possible explanations**
Viterbi = **best single explanation**

So `logsumexp` maintains the probabilistic meaning of an HMM.




Try running the forward algorithm on our vowels/consonants model from before:

In [21]:
x = torch.stack( [torch.tensor(encode("cat"))] )
T = torch.tensor([3])
print(model.forward(x, T))

x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
T = torch.tensor([3,3])
print(model.forward(x, T))

tensor([[-9.6603]], device='cuda:0')
tensor([[-5.1112],
        [   -inf]], device='cuda:0')




### **1. Why do we encode characters into integers instead of feeding letters directly?**

**Significance:**
HMMs operate on **discrete symbols**, not characters or strings.
By converting `"cat"` → `[2,0,19]`, we map letters into indices so they act as **observable emission symbols**.

**Why not feed characters?**
Characters have no numerical meaning; the HMM needs integer categories to index:

* emission probabilities `P(x | state)`
* transition matrices
* priors

This step mirrors how NLP models typically use **tokenization**.

---

### **2. Why did we *hard-wire* the model parameters instead of training them?**

**Significance:**
This demo is intended to **illustrate the structure and logic of an HMM**, not how it learns.

Hard-wiring lets you:

* **visually understand** how priors, transitions, and emissions influence probability
* **enforce linguistic constraints** (like vowel/consonant alternation)
* avoid randomness from training that can hide the intended behavior

**Why not train?**
Training would produce arbitrary parameter values unless the corpora contained perfect vowel–consonant alternation, making the conceptual lesson weaker.

---

### **3. Why force the emission matrix to allow only vowel–state emissions and consonant–state emissions?**

**Significance:**
This shows **how HMM states represent *latent categories*** — here:

* **State 0 → consonant-emitter**
* **State 1 → vowel-emitter**

By putting `-inf` for invalid symbols, we force:

```
P(vowel | consonant-state) = 0
P(consonant | vowel-state) = 0
```

**Why not allow soft probabilities?**
Because the purpose is to **illustrate deterministic constraints inside an HMM**.
Soft probabilities would dilute the structure and make the alternation pattern less clear.

---

### **4. Why restrict the transition matrix to ONLY vowel→consonant and consonant→vowel?**

**Significance:**
This enforces an **alternating HMM**:

```
C → V → C → V → ...
```

This helps students clearly see how transitions control **sequence structure**, independent of emissions.

**Why not allow C→C or V→V transitions?**
Because then the model could generate arbitrary sequences and you lose the clean, interpretable pattern.

This is a teaching example of how transitions encode **grammar-like constraints**.

---

### **5. Why call `model.forward(x, T)` with tensor shapes like this?**

Example:

```python
x = torch.stack([torch.tensor(encode("cat"))])
T = torch.tensor([3])
```

**Significance:**
The forward pass computes:

* the **log-likelihood** of each sequence
* respecting:

  * priors
  * allowed transitions
  * allowed emissions
  * sequence length `T`

`x` is shaped as a **batch of sequences**, so you see how HMMs naturally support **multiple sequences**, e.g.:

```python
x = torch.stack([encode("aba"), encode("abb")])
T = [3, 3]
```

**Why not pass raw strings?**
The model needs:

* numeric emissions
* fixed-length tensors
* batch structure
  None of these exist with raw Python strings.

---

### **6. Why do we see different probabilities for sequences like “aba” vs “abb”?**

**Significance:**
Because the HMM’s forced structure influences how likely a sequence is.
Example:

```
a b a → vowel, consonant, vowel → fits perfectly
a b b → vowel, consonant, consonant → INVALID final consonant
```

So the HMM assigns:

* **high probability** to sequences that match the alternating pattern
* **zero probability** where the last emission forbids the state structure

This shows the power of HMMs in modeling **latent structure constraints**.

---

### **7. Why set priors to prefer starting in a consonant?**

**Significance:**
Most English-like words begin with consonants.
So you set:

```
P(state0 = consonant) > P(state1 = vowel)
```

**Why not keep uniform priors?**
Uniform priors hide the effect priors have on:

* forward probabilities
* decoding
* sequence likelihood

Explicit priors demonstrate how initial state assumptions affect the entire probability calculation.




When using the vowel <-> consonant HMM from above, notice that the forward algorithm returns $-\infty$ for $\mathbf{x} = \text{"abb"}$. That's because our transition matrix says the probability of vowel -> vowel and consonant -> consonant is 0, so the probability of $\text{"abb"}$ happening is 0, and thus the log probability is $-\infty$.

#### *Side note: deriving the forward algorithm*

If you're interested in understanding how the forward algorithm actually computes $p(\mathbf{x})$, read this section; if not, skip to the next part on "Problem 2" (finding the most likely state sequence).



To derive the forward algorithm, start by deriving the forward variable:

$
\begin{align}
    \alpha_{s,t} &= p(x_1, x_2, \dots, x_t, z_t=s) \\
     &= p(x_t | x_1, x_2, \dots, x_{t-1}, z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s)  \\
    &= p(x_t | z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s) \\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s', z_t = s) \right)\\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(z_t = s | x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \cdot p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \right)\\
    &= \underbrace{p(x_t | z_t = s)}_{\text{emission model}} \cdot \left( \sum_{s'} \underbrace{p(z_t = s | z_{t-1}=s')}_{\text{transition model}} \cdot \underbrace{p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s')}_{\text{forward variable for previous timestep}} \right)\\
    &= b_s(x_t) \cdot \left( \sum_{s'} A_{s, s'} \cdot \alpha_{s',t-1} \right)
\end{align}
$

I'll explain how to get to each line of this equation from the previous line.

Line 1 is the definition of the forward variable $\alpha_{s,t}$.

Line 2 is the chain rule ($p(A,B) = p(A|B) \cdot p(B)$, where $A$ is $x_t$ and $B$ is all the other variables).

In Line 3, we apply Assumption 2: the probability of observation $x_t$ depends only on the current state $z_t$.

In Line 4, we marginalize over all the possible states in the previous timestep $t-1$.

In Line 5, we apply the chain rule again.

In Line 6, we apply Assumption 1: the current state depends only on the previous state.

In Line 7, we substitute in the emission probability, the transition probability, and the forward variable for the previous timestep, to get the complete recursion.

The formula above can be used for $t = 2 \rightarrow T$. At $t=1$, there is no previous state, so instead of the transition matrix $A$, we use the state priors $\pi$, which tell us the probability of starting in each state. Thus for $t=1$, the forward variables are computed as follows:

$$\begin{align}
\alpha_{s,1} &= p(x_1, z_1=s) \\
  &= p(x_1 | z_1 = s) \cdot p(z_1 = s)  \\
&= b_s(x_1) \cdot \pi_s
\end{align}$$

Finally, to compute $p(\mathbf{x}) = p(x_1, x_2, \dots, x_T)$, we marginalize over $\alpha_{s,T}$, the forward variables computed in the last timestep:

$$\begin{align*}
p(\mathbf{x}) &= \sum_{s} p(x_1, x_2, \dots, x_T, z_T = s) \\
&= \sum_{s} \alpha_{s,T}
\end{align*}$$

You can get from this formulation to the log domain formulation by taking the log of the forward variable, and using these identities:
- $\text{log }(a \cdot b) = \text{log }a + \text{log }b$
- $\text{log }(a + b) = \text{log }(e^{\text{log }a} + e^{\text{log }b}) = \text{logsumexp}(\text{log }a, \text{log }b)$

### Problem 2: How do we compute $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x})$?

Given an observation sequence $\mathbf{x}$, we may want to find the most likely sequence of states that could have generated $\mathbf{x}$. (Given the sequence of selfies, we want to infer what cities the friend visited.) In other words, we want $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x})$.

We can use Bayes' rule to rewrite this expression:
$$\begin{align*}
    \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x}) &= \underset{\mathbf{z}}{\text{argmax }} \frac{p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})}{p(\mathbf{x})} \\
    &= \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})
\end{align*}$$

Hmm! That last expression, $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$, looks suspiciously similar to the intractable expression we encountered before introducing the forward algorithm, $\underset{\mathbf{z}}{\sum} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$.

And indeed, just as the intractable *sum* over all $\mathbf{z}$ can be implemented efficiently using the forward algorithm, so too this intractable *argmax* can be implemented efficiently using a similar divide-and-conquer algorithm: the legendary Viterbi algorithm!

________

<u><b>The Viterbi Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,1} := b_s(x_1) \cdot \pi_s$\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,1} := 0$
>
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,t} := b_s(x_t) \cdot \left( \underset{s'}{\text{max }} A_{s, s'} \cdot \delta_{s',t-1} \right)$\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,t} := \underset{s'}{\text{argmax }} A_{s, s'} \cdot \delta_{s',t-1}$
>
> $z_T^* := \underset{s}{\text{argmax }} \delta_{s,T}$\
> for $t = T-1 \rightarrow 1$:\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_{t}^* := \psi_{z_{t+1}^*,t+1}$
>
> $\mathbf{z}^* := \{z_{1}^*, \dots, z_{T}^* \}$\
return $\mathbf{z}^*$
________

The Viterbi algorithm looks somewhat gnarlier than the forward algorithm, but it is essentially the same algorithm, with two tweaks: 1) instead of taking the sum over previous states, we take the max; and 2) we record the argmax of the previous states in a table, and loop back over this table at the end to get $\mathbf{z}^*$, the most likely state sequence. (And like the forward algorithm, we should run the Viterbi algorithm in the log domain for better numerical stability.)

Let's add the Viterbi algorithm to our PyTorch model:

In [22]:
def viterbi(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)
  Find argmax_z log p(x|z) for each (x) in the batch.
  """
  if self.is_cuda:
    x = x.cuda()
    T = T.cuda()

  batch_size = x.shape[0]; T_max = x.shape[1]
  log_state_priors = torch.nn.functional.log_softmax(self.unnormalized_state_priors, dim=0)
  log_delta = torch.zeros(batch_size, T_max, self.N).float()
  psi = torch.zeros(batch_size, T_max, self.N).long()
  if self.is_cuda:
    log_delta = log_delta.cuda()
    psi = psi.cuda()

  log_delta[:, 0, :] = self.emission_model(x[:,0]) + log_state_priors
  for t in range(1, T_max):
    max_val, argmax_val = self.transition_model.maxmul(log_delta[:, t-1, :])
    log_delta[:, t, :] = self.emission_model(x[:,t]) + max_val
    psi[:, t, :] = argmax_val

  # Get the log probability of the best path
  log_max = log_delta.max(dim=2)[0]
  best_path_scores = torch.gather(log_max, 1, T.view(-1,1) - 1)

  # This next part is a bit tricky to parallelize across the batch,
  # so we will do it separately for each example.
  z_star = []
  for i in range(0, batch_size):
    z_star_i = [ log_delta[i, T[i] - 1, :].max(dim=0)[1].item() ]
    for t in range(T[i] - 1, 0, -1):
      z_t = psi[i, t, z_star_i[0]].item()
      z_star_i.insert(0, z_t)

    z_star.append(z_star_i)

  return z_star, best_path_scores # return both the best path and its log probability

def transition_model_maxmul(self, log_alpha):
  log_transition_matrix = torch.nn.functional.log_softmax(self.unnormalized_transition_matrix, dim=0)

  out1, out2 = maxmul(log_transition_matrix, log_alpha.transpose(0,1))
  return out1.transpose(0,1), out2.transpose(0,1)

def maxmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Similar to the log domain matrix multiplication,
	this computes out_{i,j} = max_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	log_A_expanded = torch.stack([log_A] * p, dim=2)
	log_B_expanded = torch.stack([log_B] * m, dim=0)

	elementwise_sum = log_A_expanded + log_B_expanded
	out1,out2 = torch.max(elementwise_sum, dim=1)

	return out1,out2

TransitionModel.maxmul = transition_model_maxmul
HMM.viterbi = viterbi



### **1. Why do we use log probabilities in Viterbi instead of raw probabilities?**

**Significance:**
Multiplying many small probabilities quickly underflows to zero.
Viterbi uses **addition of log-probs instead of multiplication of probs**, which is numerically stable.

**Why not stay in probability space?**

* HMM sequences quickly get tiny: (10^{-30}), (10^{-50}), etc.
* Logs prevent collapse to zero
* Max over log-probs = max over probs (monotonic transformation)

---

### **2. Why does Viterbi use `max` (argmax path) while forward uses `sum` (total probability)?**

**Significance:**
Forward computes:

> **Likelihood of the sequence** over *all* state paths.

Viterbi computes:

> **Most likely hidden-state path**, not its total probability.

**Why not use sum here?**
Sum answers *“How likely is the sequence?”*
Max answers *“Which sequence of states best explains it?”*
For decoding → max is the correct operation.

---

### **3. Why do we use a custom `maxmul()` instead of PyTorch matrix multiply?**

**Significance:**
Typical matrix multiplication uses:

```
sum_k A[i,k] * B[k,j]
```

Viterbi needs:

```
max_k A[i,k] + B[k,j]
```

This is **max-plus semiring** (a different algebra).

Standard matmul cannot do that.

Using a custom `maxmul()` explicitly demonstrates to learners:

* HMM algorithms operate in special semirings
* Viterbi uses max-plus
* Forward uses log-sum-exp

This is an educational moment: *same HMM, different algebra → different inference*.

---

### **4. Why do we store `psi` (the backpointer matrix)?**

**Significance:**
`psi[t][j]` tells you:

> *“From which previous state did we come to reach state `j` at time `t` with the highest probability?”*

Without `psi`, you can compute the score of the best path, but **not the actual path**.

**Why not recompute path afterward?**
Because recomputing loses all intermediate decisions, impossible without backpointers.

---

### **5. Why do we manually backtrack through time in a loop instead of parallelizing?**

**Significance:**
Parallelizing backtracking is hard because:

* each step depends on the previous state’s choice
* backtracking is inherently sequential
  (you can only know the best state at `t-1` after knowing the best at `t`)

For teaching purposes, this loop makes the logic transparent.

**Why not vectorize anyway?**
It complicates the code and hides the conceptual flow.

---

### **6. Why do we apply emission probabilities before transitions at the first step?**

Code:

```python
log_delta[:, 0, :] = emission(x[:,0]) + log_state_priors
```

**Significance:**
At time 0, no transitions have occurred.
The best score for being in state `i` is:

```
P(z0 = i) + P(x0 | zi)
```

Teaching point: **first step = priors + first observation**.

**Why not include transition from a dummy start state?**
That’s equivalent, but increases conceptual overhead for students.

---

### **7. Why does “aba” give a valid path but “abb” fails or gives worse score?**

**Significance:**
Your HMM enforces alternating vowel/consonant states:

```
C → V → C → V ...
```

“aba”

```
a (vowel)  
b (consonant)  
a (vowel)
→ perfectly fits transitions + emissions
```

“abb”

```
a (vowel)
b (consonant)
b (consonant) → forbidden emission → -inf score
```

Thus Viterbi correctly returns:

* a valid state path for “aba”
* impossible/low-probability path for “abb”

This demonstrates how **HMM constraints produce structured decoding**.

---

### **8. Why is `best_path_scores` taken from the last timestep using gather?**

**Significance:**
If sequences have different lengths, Viterbi must take:

```
score of best state at time T[i] - 1
```

not at `T_max - 1`.

This teaches:

* HMMs support variable-length sequences
* we score only up to the true end, not the padded part

**Why not use a fixed-length sequence?**
Because batching variable-length sequences is standard in NLP.

---

### **9. Why run Viterbi on a batch (two words at once) instead of one word at a time?**

**Significance:**
Efficient NLP models process multiple sequences simultaneously.
Running:

```python
["aba", "abb"]
```

together demonstrates:

* vectorized HMM inference
* shared parameters across sequences
* practical batching behavior

**Why not run separately?**
Less realistic and slower in any real system.

---

### **10. Why return both `(z_star, best_path_scores)` instead of only the path?**

**Significance:**
Viterbi decoding answers two questions:

1. **Which path is most probable?** → `z_star`
2. **How probable is that path?** → `best_path_scores`

This matters for:

* comparing sequences
* confidence scoring
* ranking alternative segmentations or spellings

**Why not return only the path?**
You lose important interpretability and scoring information.




Try running Viterbi on an input sequence, given the vowel/consonant HMM:

In [23]:
x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
T = torch.tensor([3,3])
print(model.viterbi(x, T))

([[1, 0, 1], [1, 0, 0]], tensor([[-5.1112],
        [   -inf]], device='cuda:0'))


For $\mathbf{x} = \text{"aba"}$, the Viterbi algorithm returns $\mathbf{z}^* = \{1,0,1\}$. This corresponds to "vowel, consonant, vowel" according to the way we defined the states above, which is correct for this input sequence. Yay!

For $\mathbf{x} = \text{"abb"}$, the Viterbi algorithm still returns a $\mathbf{z}^*$, but we know this is gibberish because "vowel, consonant, consonant" is impossible under this HMM, and indeed the log probability of this path is $-\infty$.

Let's compare the "forward score" (the log probability of all possible paths, returned by the forward algorithm) with the "Viterbi score" (the log probability of the maximum likelihood path, returned by the Viterbi algorithm):

In [24]:
print(model.forward(x, T))
print(model.viterbi(x, T)[1])

tensor([[-5.1112],
        [   -inf]], device='cuda:0')
tensor([[-5.1112],
        [   -inf]], device='cuda:0')


The two scores are the same! That's because in this instance there is only one possible path through the HMM, so the probability of the most likely path is the same as the sum of the probabilities of all possible paths.

In general, though, the forward score and Viterbi score will always be somewhat close. This is because of a property of the $\text{logsumexp}$ function: $\text{logsumexp}(\mathbf{x}) \approx \max (\mathbf{x})$. ($\text{logsumexp}$ is sometimes referred to as the "smooth maximum" function.)

In [25]:
x = torch.tensor([1., 2., 3.])
print(x.max(dim=0)[0])
print(x.logsumexp(dim=0))

tensor(3.)
tensor(3.4076)


### Problem 3: How do we train the model?





Earlier, we hard-coded an HMM to have certain behavior. What we would like to do instead is have the HMM learn to model the data on its own. And while it is possible to use supervised learning with an HMM (by hard-coding the emission model or the transition model) so that the states have a particular interpretation, the really cool thing about HMMs is that they are naturally unsupervised learners, so they can learn to use their different states to represent different patterns in the data, without the programmer needing to indicate what each state means.

Like many machine learning models, an HMM can be trained using maximum likelihood estimation, i.e.:

$$\theta^* = \underset{\theta}{\text{argmin }} -\sum_{\mathbf{x}^i}\text{log }p_{\theta}(\mathbf{x}^i)$$

where $\mathbf{x}^1, \mathbf{x}^2, \dots$ are training examples.

The standard method for doing this is the Expectation-Maximization (EM) algorithm, which for HMMs is also called the "Baum-Welch" algorithm. In EM training, we alternate between an "E-step", where we estimate the values of the latent variables, and an "M-step", where the model parameters are updated given the estimated latent variables. (Think $k$-means, where you guess which cluster each data point belongs to, then reestimate where the clusters are, and repeat.) The EM algorithm has some nice properties: it is guaranteed at each step to decrease the loss function, and the E-step and M-step may have an exact closed form solution, in which case no pesky learning rates are required.

But because the HMM forward algorithm is differentiable with respect to all the model parameters, we can also just take advantage of automatic differentiation methods in libraries like PyTorch and try to minimize $-\text{log }p_{\theta}(\mathbf{x})$ directly, by backpropagating through the forward algorithm and running stochastic gradient descent. That means we don't need to write any additional HMM code to implement training: `loss.backward()` is all you need.

Here we will implement SGD training for an HMM in PyTorch. First, some helper classes:

In [26]:
import torch.utils.data
from collections import Counter
from sklearn.model_selection import train_test_split

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, lines):
    self.lines = lines # list of strings
    collate = Collate() # function for generating a minibatch from strings
    self.loader = torch.utils.data.DataLoader(self, batch_size=1024, num_workers=1, shuffle=True, collate_fn=collate)

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx):
    line = self.lines[idx].lstrip(" ").rstrip("\n").rstrip(" ").rstrip("\n")
    return line

class Collate:
  def __init__(self):
    pass

  def __call__(self, batch):
    """
    Returns a minibatch of strings, padded to have the same length.
    """
    x = []
    batch_size = len(batch)
    for index in range(batch_size):
      x_ = batch[index]

      # convert letters to integers
      x.append(encode(x_))

    # pad all sequences with 0 to have same length
    x_lengths = [len(x_) for x_ in x]
    T = max(x_lengths)
    for index in range(batch_size):
      x[index] += [0] * (T - len(x[index]))
      x[index] = torch.tensor(x[index])

    # stack into single tensor
    x = torch.stack(x)
    x_lengths = torch.tensor(x_lengths)
    return (x,x_lengths)

Let's load some training/testing data. By default, this will use the unix "words" file, but you could also use your own text file.



## **1. Why do we print both `model.forward(x, T)` and `model.viterbi(x, T)[1]`?**

### **Significance (HMM Concept)**

You are demonstrating that:

* **Forward algorithm** computes
  → *total probability of all state paths producing the sequence*.

* **Viterbi algorithm** computes
  → *probability of only the best single state path*.

So the forward log-probability is always **≥** the Viterbi log-score (log-sum-exp ≥ max).
Printing them together visually proves this property.

### **Why not print only one?**

Because students often confuse “likelihood of the sequence” with “likelihood of the best path.”
Seeing both reinforces the conceptual distinction.

---

## **2. Why do we compute both `x.max(dim=0)` and `x.logsumexp(dim=0)`?**

### **Significance (HMM Math Insight)**

You are demonstrating the key algebraic difference:

* Viterbi uses **max** (max-plus semiring)
* Forward uses **logsumexp** (sum-product semiring)

This simple vector example:

```python
x = [1, 2, 3]
```

lets students see:

* Max = 3
* Log-sum-exp = log(e¹ + e² + e³) > 3

This illustrates why forward scores > Viterbi scores always.

### **Why not explain using only theory?**

This tiny concrete example makes the connection obvious and intuitive.

---

## **3. Why do we pad sequences inside `Collate()` instead of letting PyTorch handle variable lengths automatically?**

### **Significance (HMM Implementation Detail)**

Your HMM implementation expects a tensor shaped:

```
batch_size × T_max
```

because:

* Forward algorithm needs uniform time steps
* Viterbi needs a consistent DP table shape
* You manually track the true lengths via `x_lengths`

### **Why not use PyTorch’s PackedSequence / RNN utilities?**

Because this is a *hand-built* HMM, not an RNN.
Packing/unpacking is unnecessary and would hide important details from students.

Padding + explicit lengths is the standard method in *classic HMM implementations*.

---

## **4. Why does `Collate()` both pad **and** return `x_lengths` (T)?**

### **Significance (HMM Requirement)**

Even though you pad sequences, the forward and Viterbi algorithms must only use the true length.

If you didn’t pass `T`, the model would try to interpret padded zeros as real emissions, which ruins:

* log-probabilities
* state decoding
* evaluation results

### **Why not mask instead of using `T`?**

Masking complicates the code.
Using `T` keeps the algorithms clean:

* forward → `log_probs = torch.gather(..., T-1)`
* Viterbi → backtracking starts at `T-1`

This is the simplest correct approach.

---

## **5. Why does `__getitem__` strip whitespace before encoding the line?**

### **Significance**

Many datasets contain:

* leading spaces
* trailing spaces
* stray newline characters

If you don’t strip:

* HMM will treat spaces as extra tokens
* encoded sequences will be wrong
* emissions matrices become polluted

### **Why not let the encoder ignore unknown chars?**

Encoding unknown characters silently hides mistakes.
Stripping ensures clean, consistent training data.

---

## **6. Why stack two sequences (`aba`, `abb`) into the same batch?**

### **Significance**

Batching demonstrates how HMMs scale.
Your forward and Viterbi implementations are **vectorized across batch dimension**, so they:

* share transition/emission parameters
* compute in parallel
* mimic real HMM usage in NLP tasks

### **Why not run sequences independently?**

It hides the efficiency benefits of HMMs and breaks the demo’s focus on batching.

---

## **7. Why is padding done using the integer `0` specifically?**

### **Significance**

`0` corresponds to letter `'a'` in your alphabet.
But in HMM training, that doesn’t matter because:

* you always **mask out padded positions using `T`**
* forward and Viterbi never look at padded timesteps
* emissions for the padded section are irrelevant

### **Why not use a special PAD token?**

Because the model never actually reads padded timesteps—`T` handles it.
This avoids enlarging the alphabet.

---

## **8. Why is the `TextDataset` holding a `DataLoader` inside it?**

### **Significance**

This design makes your training loop extremely simple:

```python
for x, T in dataset.loader:
    ...
```

It packages:

* the data
* batching
* padding
* encoding

into a single object.

### **Why not put the DataLoader outside?**

For a classroom demo, embedding it inside keeps code clean and easy to follow.

---

## **9. Why do we use `Counter` and train/test split but not shown in this snippet?**

### **Significance**

You are preparing the dataset to later show:

* distribution of characters
* splitting of corpus into training/testing for likelihood evaluation
* classic NLP preprocessing

This lays groundwork for:

* learning HMM parameters
* evaluating perplexity
* computing held-out likelihood

---

## **10. Why are we printing outputs after each step in this notebook-style code?**

### **Significance**

This is a **didactic style**:

* Show intermediate numeric results
* Reinforce understanding of each algorithmic step
* Compare max vs log-sum-exp
* Compare forward vs Viterbi
* See batching effects directly

### **Why not wrap everything in functions and hide output?**

Because transparency is crucial for teaching how HMM inference works.




In [27]:
!wget https://raw.githubusercontent.com/lorenlugosch/pytorch_HMM/master/data/train/training.txt

filename = "training.txt"

with open(filename, "r") as f:
  lines = f.readlines() # each line of lines will have one word

alphabet = list(Counter(("".join(lines))).keys())
train_lines, valid_lines = train_test_split(lines, test_size=0.1, random_state=42)
train_dataset = TextDataset(train_lines)
valid_dataset = TextDataset(valid_lines)

M = len(alphabet)

--2025-11-23 04:43:58--  https://raw.githubusercontent.com/lorenlugosch/pytorch_HMM/master/data/train/training.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2493109 (2.4M) [text/plain]
Saving to: ‘training.txt.1’

training.txt.1      100%[===================>]   2.38M  --.-KB/s    in 0.04s   

2025-11-23 04:43:58 (66.3 MB/s) - ‘training.txt.1’ saved [2493109/2493109]



We will use a Trainer class for training and testing the model:



In [28]:
from tqdm import tqdm # for displaying progress bar

class Trainer:
  def __init__(self, model, lr):
    self.model = model
    self.lr = lr
    self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=0.00001)

  def train(self, dataset):
    train_loss = 0
    num_samples = 0
    self.model.train()
    print_interval = 50
    for idx, batch in enumerate(tqdm(dataset.loader)):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      train_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        for _ in range(5):
          sampled_x, sampled_z = self.model.sample()
          print(decode(sampled_x))
          print(sampled_z)
    train_loss /= num_samples
    return train_loss

  def test(self, dataset):
    test_loss = 0
    num_samples = 0
    self.model.eval()
    print_interval = 50
    for idx, batch in enumerate(dataset.loader):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      test_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        sampled_x, sampled_z = self.model.sample()
        print(decode(sampled_x))
        print(sampled_z)
    test_loss /= num_samples
    return test_loss

Finally, initialize the model and run the main training loop. Every 50 batches, the code will produce a few samples from the model. Over time, these samples should look more and more realistic.

In [29]:
# Initialize model
model = HMM(N=64, M=M)

# Train the model
num_epochs = 10
trainer = Trainer(model, lr=0.01)

for epoch in range(num_epochs):
        print("========= Epoch %d of %d =========" % (epoch+1, num_epochs))
        train_loss = trainer.train(train_dataset)
        valid_loss = trainer.test(valid_dataset)

        print("========= Results: epoch %d of %d =========" % (epoch+1, num_epochs))
        print("train loss: %.2f| valid loss: %.2f\n" % (train_loss, valid_loss) )

========= Epoch 1 of 10 =========


  0%|          | 1/208 [00:00<00:40,  5.09it/s]

loss: 38.02503204345703
FbYkdNLhKk
[1, 51, 8, 61, 49, 35, 22, 63, 28, 12]
YlqRm-yXBU
[54, 26, 27, 28, 22, 40, 32, 4, 51, 44]
ZXnvXoCQDz
[22, 43, 42, 58, 4, 39, 9, 41, 13, 46]
eeWqolQGVN
[52, 44, 25, 31, 40, 0, 31, 51, 51, 37]
DWUYxpvudP
[61, 46, 44, 34, 33, 33, 31, 14, 48, 37]


 25%|██▌       | 53/208 [00:03<00:09, 16.25it/s]

loss: 32.93939971923828
UgoasnNGni
[1, 50, 32, 42, 20, 32, 20, 16, 42, 42]

piLespevY
[2, 43, 61, 22, 7, 56, 14, 36, 17, 20]
paieohscKs
[47, 49, 2, 40, 2, 6, 26, 32, 18, 55]
BNNH-eFaTK
[30, 4, 17, 30, 37, 15, 33, 0, 28, 53]
tnosAtoagj
[47, 2, 62, 26, 19, 3, 7, 56, 55, 10]


...

loss: 23.52859878540039
prasoptral
[47, 43, 20, 29, 16, 14, 40, 43, 16, 36]
sipushyder
[60, 20, 14, 46, 26, 6, 61, 0, 15, 35]
pirisiapii
[47, 33, 35, 32, 26, 32, 16, 14, 43, 20]
Nestyrrope
[4, 7, 26, 40, 33, 35, 43, 58, 14, 15]
trogissito
[47, 43, 58, 50, 32, 26, 29, 32, 40, 58]


 98%|█████████▊| 203/208 [00:12<00:00, 16.83it/s]

loss: 23.023834228515625
hmentineli
[6, 49, 16, 42, 40, 32, 48, 15, 36, 32]
pyunnierwh
[47, 1, 54, 2, 48, 32, 16, 35, 9, 6]
thettorolm
[9, 6, 7, 42, 40, 15, 35, 33, 36, 49]
nanistPung
[4, 20, 42, 32, 26, 29, 47, 54, 2, 50]
moirrorome
[4, 51, 33, 35, 43, 58, 43, 58, 49, 7]


100%|██████████| 208/208 [00:12<00:00, 16.48it/s]


loss: 23.31765365600586
sprierycop
[60, 14, 43, 20, 15, 35, 28, 45, 16, 14]
========= Results: epoch 10 of 10 =========
train loss: 23.30| valid loss: 23.32



You may wish to try different values of $N$ and see what the impact on sample quality is.

In [30]:
x = torch.tensor(encode("quack")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T))

x = torch.tensor(encode("quick")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T))

x = torch.tensor(encode("qurck")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T)) # should have lower probability---in English only vowels follow "qu"

x = torch.tensor(encode("qiick")).unsqueeze(0)
T = torch.tensor([5])
print(model.viterbi(x,T)) # should have lower probability---in English only "u" follows "q"


([[30, 44, 17, 45, 12]], tensor([[-13.7824]], device='cuda:0', grad_fn=<GatherBackward0>))
([[30, 44, 32, 45, 12]], tensor([[-10.9867]], device='cuda:0', grad_fn=<GatherBackward0>))
([[30, 44, 35, 45, 12]], tensor([[-19.2461]], device='cuda:0', grad_fn=<GatherBackward0>))
([[30, 44, 32, 45, 12]], tensor([[-19.3918]], device='cuda:0', grad_fn=<GatherBackward0>))


In [31]:
import torch

def show_prob(word):
    x = torch.tensor(encode(word)).unsqueeze(0)
    T = torch.tensor([len(word)])

    path, logp = model.viterbi(x, T)

    # Convert log-prob → normal probability
    prob = torch.exp(logp).item()

    print(f"word: {word}")
    print(f"best state path: {path[0]}")
    print(f"log probability: {logp.item():.4f}")
    print(f"normal probability: {prob:.10f}\n")  # 10 decimal places


# Run your examples
show_prob("quack")
show_prob("quick")
show_prob("qurck")  # should be low
show_prob("qiick")  # should be low


word: quack
best state path: [30, 44, 17, 45, 12]
log probability: -13.7824
normal probability: 0.0000010337

word: quick
best state path: [30, 44, 32, 45, 12]
log probability: -10.9867
normal probability: 0.0000169247

word: qurck
best state path: [30, 44, 35, 45, 12]
log probability: -19.2461
normal probability: 0.0000000044

word: qiick
best state path: [30, 44, 32, 45, 12]
log probability: -19.3918
normal probability: 0.0000000038





## **1. Why do we build the alphabet using `Counter(("".join(lines)))` instead of reusing the earlier alphabet = 'a'..'z'?**

### **Significance (HMM + Data-Driven Modeling)**

This time the HMM is being **trained on real text**, not hard-coded toy examples.

So we extract the alphabet *from the actual dataset itself*, meaning:

* If the corpus contains accented letters, apostrophes, hyphens — they get included
* If the dataset excludes some letters, the model won’t waste emission parameters on unused symbols

### **Why not use fixed 26-letters?**

Because real data vocabulary may be:

* smaller
* larger
* domain-specific

A data-driven alphabet prevents building an incorrect emission matrix.

---

## **2. Why do we train the HMM with `N=64` states? Why 64 specifically?**

### **Significance (What HMM states are actually doing)**

In a real HMM, the **states are not vowels/consonants** anymore — they are **latent linguistic clusters**.

64 states gives the model enough “expressive capacity” to learn:

* onset vs nucleus vs coda patterns
* consonant types (plosives, fricatives, liquids…)
* vowel classes
* clusters like “qu”, “ck”, “ng”, “sh”, “th”
* position-dependent variants (initial-q, mid-q, final-q, etc.)

### **Why not use 2 or 3 states?**

Because natural language morphology is too rich — low-state HMMs cannot capture even basic English orthography patterns.

### **Why not use 256 states?**

Training becomes unstable and slow for this demo; 64 is a sweet spot of expressive power vs simplicity.

---

## **3. Why is the training loss defined as `loss = -log_probs.mean()`?**

### **Significance (Maximum Likelihood for HMMs)**

Forward returns **log p(x)**, so maximizing likelihood means minimizing **negative log-likelihood**.

This corresponds to MLE:

```
maximize    ∑ log pθ(x)
equivalent to minimize  -∑ log pθ(x)
```

### **Why not cross-entropy?**

Cross-entropy applies to classification targets.
Here, the target is an entire *sequence*, so the forward probability is correct.

---

## **4. Why show sampled sequences during training (`model.sample()`) every 50 batches?**

### **Significance (Interpreting HMM learning qualitatively)**

Sampling shows **what the model believes English words look like** at the current epoch.

You can visually track:

* random gibberish at epoch 1
* emergence of vowel/consonant balance
* discovery of frequent patterns:

  * “qu”
  * “ck”
  * double consonants
  * English-like endings (“ing”, “ed”, “er”)

Sampling is the **best sanity check** to ensure training is working.

### **Why not wait until the end?**

Intermediate monitoring is essential when teaching or debugging sequence models.

---

## **5. Why use Adam optimizer with weight decay?**

### **Significance (Stabilizing HMM Learning)**

HMMs trained by gradient descent can suffer from:

* very sharp posteriors
* collapse of states
* overfitting emission distributions
* becoming degenerate (one state dominates)

Weight decay acts like **soft regularization** on:

* state priors
* transitions
* emissions

Adam helps with noisy gradients from variable-length minibatches.

### **Why not use Baum–Welch (EM)?**

Because:

* EM is harder to implement from scratch
* cannot run on GPUs cleanly
* gradient-based HMMs are easier to integrate into PyTorch’s ecosystem
* pedagogically simpler for showing training loops

---

## **6. Why do we split 90% training / 10% validation?**

### **Significance (True Evaluation of Generative Models)**

Validation loss measures:

* generalization of the HMM
* whether the model has learned real English structure instead of memorization

Overfitting appears when:

* train loss ↓
* validation loss ↑

Classic for generative sequence models.

---

## **7. Why does the model correctly score `"quack"` or `"quick"` higher than `"qurck"` or `"qiick"`?**

### **Significance (HMM’s ability to learn orthographic dependencies)**

During training, the model learns high transition+emission probability for:

```
q → u
```

Because in English, almost every "q" is followed by "u".

Similarly it learns:

* “ck”
* “qu”
* “ing”
* “sh”, “ch”
* vowel/consonant alternation tendencies

### **Why not use RNNs to learn this?**

Because you are demonstrating that **even simple HMMs** can capture surprising amounts of structure.

---

## **8. Why do we wrap everything in a `Trainer` class instead of writing a single loop?**

### **Significance (Clean Architecture)**

This emphasizes modularity:

* `trainer.train(dataset)`
* `trainer.test(dataset)`

In real machine learning pipelines, such separation is important.

### **Why not use Lightning or high-level trainers?**

Because you want students to see **exactly how training works**, step-by-step.

---

## **9. Why do we repeatedly evaluate validation loss every epoch?**

### **Significance**

Monitoring validation loss ensures the HMM:

* is not collapsing
* is improving English modeling
* is not diverging
* is not overfitting

Generative models MUST be monitored because they can silently degrade.

---

## **10. Why does `viterbi("qurck")` have lower probability?**

### **Significance (Sequence-Level Reasoning)**

Viterbi detects the **best hidden state path**.
For irregular words like “qurck”, the emission/transition structure must force unlikely sequences of states, lowering score.

This directly shows students the interpretability of HMMs:

* Viterbi path is inspectable
* You can see state clusters that correspond to linguistic categories
* You observe where the sequence violates English patterns




## Conclusion

HMMs used to be very popular in natural language processing, but they have largely been overshadowed by neural network models like RNNs and Transformers. Still, it is fun and instructive to study the HMM; some commonly used machine learning techniques like [Connectionist Temporal Classification](https://www.cs.toronto.edu/~graves/icml_2006.pdf) are inspired by HMM methods. HMMs are [still used in conjunction with neural networks in speech recognition](https://arxiv.org/abs/1811.07453), where the assumption of a one-hot state makes sense for modelling phonemes, which are spoken one at a time.

## Acknowledgments

This notebook is based partly on Lawrence Rabiner's excellent article "[A Tutorial on Hidden Markov Models and Selected Applications in Speech Recognition](https://www.cs.cmu.edu/~cga/behavior/rabiner1.pdf)", which you may also like to check out. Thanks also to Dima Serdyuk and Kyle Gorman for their feedback on the draft.